In [8]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2


from sklearn import model_selection
from tensorflow import keras
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from tensorflow.keras.applications import *
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten,Conv2D, MaxPooling2D,BatchNormalization

%matplotlib inline

In [ ]:
labels=os.listdir(r'C:\Users\laala\Downloads\Mini_Project\IMAGES')
print(labels)
all_files=[]
for item in labels:
    files = os.listdir('C:/Users/laala/Downloads/Mini_Project/IMAGES'+'/'+item)
    for i in files:
        all_files.append((item,str(item+'/'+i)))
all_files
data=pd.DataFrame(data=all_files,columns=['Labels','Image'])
data
images=[]
label=[]
path = 'C:/Users/laala/Downloads/Mini_Project/IMAGES'

for i in range(0,len(all_files)):
    filepath=str(path+'/'+all_files[i][1])
    img=(cv2.imread(filepath))
    img=cv2.resize(img,(227,227))
    images.append(img)
    label.append(all_files[i][0])
data
images=np.array(images)
images.shape
y=pd.get_dummies(data.Labels)
print(y)
input_shape=(227,227,3)
model=Sequential()
model.add(Conv2D(filters=96, input_shape=(227,227,3), kernel_size=(11,11),strides=(4,4)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.summary()




In [ ]:
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
model.add(BatchNormalization())
model.summary()


In [ ]:
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
model.add(BatchNormalization())
model.summary()


In [ ]:
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.summary()



In [ ]:


# Define the CNN architecture
model = tf.keras.Sequential([
    layers.Conv2D(10, (9, 9), activation='relu', input_shape=(336, 448, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(10, (6, 6), activation='relu'),
    layers.MaxPooling2D((3, 3)),
    layers.Flatten(),
    layers.Dense(2, activation='softmax')
])


######################################################################################
###Memory module and bottleneck code
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation, Flatten, Dense

# Define the CNN model
model = Sequential()

# Add the input layer
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(227,227, 3)))

# Add the first max pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add the first batch normalization layer
model.add(BatchNormalization())

# Add the first ReLU activation layer
model.add(Activation('relu'))

# Add the second max pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add the second batch normalization layer
model.add(BatchNormalization())

# Add the second ReLU activation layer
model.add(Activation('relu'))

# Add the bottleneck layer
model.add(Conv2D(64, (1,1), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (1,1), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
# Add the third ReLU activation layer
model.add(Activation('relu'))

# Add the flattening layer
model.add(Flatten())

# Add the fully connected layer
model.add(Dense(128))

# Add the third batch normalization layer
model.add(BatchNormalization())

# Add the fourth ReLU activation layer
model.add(Activation('relu'))

# Add the output layer with softmax activation function for classification
model.add(Dense(num_classes, activation='softmax'))

# Compile the model with categorical cross-entropy loss function and Adam optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint

x_train,x_test,y_train,y_test=train_test_split(images, y, shuffle=True, random_state=42, test_size=0.2)
x_test,x_val,y_test,y_val=train_test_split(x_test, y_test, shuffle=True, random_state=42, test_size=0.5 )
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.00001),loss='categorical_crossentropy',metrics=['accuracy'])
model_checkpoint=ModelCheckpoint('Model.h5', monitor='acc', mode='max', verbose=1, save_best_only=True)
history = model.fit(x_train, y_train, epochs=20)




In [ ]:
plt.plot(history.history['acc'], color='Red')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from keras.models import load_model 
model=load_model('Model.h5')
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print("Test Loss: ",test_loss)
print("Test Accuracy: ",round(test_acc*100,2),"%")
y_pred=model.predict(x_val)
metric=tf.keras.metrics.Accuracy()
metric.update_state(y_pred, y_val)
metric.result().numpy()
val_acc=round(metric.result().numpy()*100,2)
print("Accuracy of validation set is {a}%".format(a=val_acc))

In [ ]:
from google.colab import files
import cv2
import numpy as np

print("Submit your image")

flag = False
while not flag:
    uploaded = files.upload()
    for name, data in uploaded.items():
        img = cv2.imdecode(np.fromstring(data, np.uint8), cv2.IMREAD_UNCHANGED)
        if img is None:
            print("Can not find any image. Choose appropriate file")
            continue
        else:
            flag = True
            break

img.shape
img=cv2.resize(img,(227,227))
img


In [ ]:
from PIL import Image as im
image=im.fromarray(img)
image
img=img[np.newaxis,...]
img.shape
ans=model.predict(img)
ans
index=np.argmax(ans)
index
accuracy=round((np.amax(ans)*100),2)
print("There is a {a}% chance that the given shoe is {d}".format(a=accuracy, d=labels[index]))
